<a href="https://colab.research.google.com/github/guten-morgen3776/MUFG-Data-Science-Challenge-2025/blob/main/mufg_xgboost_thr_avg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold

In [3]:
train_data = pd.read_csv('/content/drive/MyDrive/signate mufg/train-3.csv')
test_data = pd.read_csv('/content/drive/MyDrive/signate mufg/test-2.csv')


In [4]:
train_data.head(3)

,id,GrossApproval,SBAGuaranteedApproval,ApprovalFiscalYear,Subprogram,InitialInterestRate,FixedOrVariableInterestInd,TermInMonths,NaicsSector,CongressionalDistrict,BusinessType,BusinessAge,RevolverStatus,JobsSupported,CollateralInd,LoanStatus
0,1,23000,7093,2020,FA$TRK (Small Loan Express),11.92,V,84,Accommodation_food services,27,CORPORATION,Unanswered,0,1,N,1
1,2,104300,83477,2020,FA$TRK (Small Loan Express),8.40,V,102,Retail trade,14,CORPORATION,"Startup, Loan Funds will Open Business",1,1,N,1
2,3,163900,53246,2020,Contract Guaranty,4.32,F,119,Construction,0,CORPORATION,Existing or more than 2 years old,1,79,Y,0


In [5]:
print(train_data.info())
print(test_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7552 entries, 0 to 7551
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          7552 non-null   int64  
 1   GrossApproval               7552 non-null   int64  
 2   SBAGuaranteedApproval       7552 non-null   int64  
 3   ApprovalFiscalYear          7552 non-null   int64  
 4   Subprogram                  7552 non-null   object 
 5   InitialInterestRate         7552 non-null   float64
 6   FixedOrVariableInterestInd  7552 non-null   object 
 7   TermInMonths                7552 non-null   int64  
 8   NaicsSector                 7552 non-null   object 
 9   CongressionalDistrict       7552 non-null   int64  
 10  BusinessType                7552 non-null   object 
 11  BusinessAge                 7552 non-null   object 
 12  RevolverStatus              7552 non-null   int64  
 13  JobsSupported               7552 

In [6]:
target = 'LoanStatus'
cat_cols = ["Subprogram", "FixedOrVariableInterestInd", "NaicsSector",
            "BusinessType", "BusinessAge", "CollateralInd"]
id_col = 'id'

In [7]:
cardinality = {col: train_data[col].nunique() for col in cat_cols}
print(cardinality)

{'Subprogram': 9, 'FixedOrVariableInterestInd': 2, 'NaicsSector': 19, 'BusinessType': 4, 'BusinessAge': 5, 'CollateralInd': 2}


In [8]:
cardinality = {col: test_data[col].nunique() for col in cat_cols}
print(cardinality)

{'Subprogram': 9, 'FixedOrVariableInterestInd': 2, 'NaicsSector': 20, 'BusinessType': 4, 'BusinessAge': 5, 'CollateralInd': 2}


In [9]:
le = LabelEncoder()
train_data['NaicsSector'] = le.fit_transform(train_data['NaicsSector'].astype(str))
test_data['NaicsSector'] = le.fit_transform(test_data['NaicsSector'].astype(str))

In [10]:
ohe_cols = ["Subprogram", "FixedOrVariableInterestInd",
            "BusinessType", "BusinessAge", "CollateralInd"]

In [11]:
train_data = train_data = pd.get_dummies(train_data, columns=ohe_cols).astype(int)
test_data = pd.get_dummies(test_data, columns=ohe_cols).astype(int)

In [12]:
train_data.head(3)

,id,GrossApproval,SBAGuaranteedApproval,ApprovalFiscalYear,InitialInterestRate,TermInMonths,NaicsSector,CongressionalDistrict,RevolverStatus,JobsSupported,...,BusinessType_CORPORATION,BusinessType_INDIVIDUAL,BusinessType_PARTNERSHIP,BusinessAge_Change of Ownership,BusinessAge_Existing or more than 2 years old,BusinessAge_New Business or 2 years or less,"BusinessAge_Startup, Loan Funds will Open Business",BusinessAge_Unanswered,CollateralInd_N,CollateralInd_Y
0,1,23000,7093,2020,11,84,0,27,0,1,...,1,0,0,0,0,0,0,1,1,0
1,2,104300,83477,2020,8,102,15,14,1,1,...,1,0,0,0,0,0,1,0,1,0
2,3,163900,53246,2020,4,119,4,0,1,79,...,1,0,0,0,1,0,0,0,0,1


In [13]:
train_data['GuaranteeRatio'] = train_data['SBAGuaranteedApproval'] / train_data['GrossApproval']
test_data['GuaranteeRatio'] = test_data['SBAGuaranteedApproval'] / test_data['GrossApproval']

In [14]:
train_data['SBAGuaranteedApproval * InitialInterestRate'] = train_data['SBAGuaranteedApproval'] * train_data['InitialInterestRate']
test_data['SBAGuaranteedApproval * InitialInterestRate'] = test_data['SBAGuaranteedApproval'] * test_data['InitialInterestRate']

In [15]:
X = train_data.drop(columns=[target,'id'])
y = train_data[target]

In [16]:
X = X.replace([np.inf, -np.inf], np.nan)


In [ ]:
SEEDS = [314, 2718, 15, 811, 73, 246]
X_test = test_data.drop(columns=['id'])
X_test = X_test.replace([np.inf, -np.inf], np.nan)
oof_preds_seeds = []
test_preds_seeds = []   # test_data がある場合のみ使う
oof_true = y.copy()
all_fold_thrs = []

for sd in SEEDS:
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=sd)
    scores = []
    oof_preds = pd.Series(0,index=y.index, dtype=float)
    test_fold_preds = []
    fold_thrs = []

    for train_idx, valid_idx in skf.split(X, y):
      X_train,X_valid = X.iloc[train_idx],X.iloc[valid_idx]
      y_train,y_valid = y.iloc[train_idx],y.iloc[valid_idx]

      model = XGBClassifier(
          objective ='binary:logistic',
          n_estimators=10000,
          learning_rate=0.05,
          eval_metric='auc',
          tree_method='hist',
          random_state=sd,
          early_stopping_rounds=100
      )
      model.fit(
          X_train,y_train,
          eval_set=[(X_valid,y_valid)],
          verbose=False
      )
      preds = model.predict_proba(X_valid,iteration_range=(0,model.best_iteration))[:,1]
      oof_preds.iloc[valid_idx] = preds
      #fold毎に閾値を最適化
      ths = np.linspace(0.05,0.95,181)
      f1s = [f1_score(y_valid, (preds > t).astype(int)) for t in ths]
      best_thr_fold = ths[int(np.argmax(f1s))]
      fold_thrs.append(best_thr_fold)

      test_fold_preds.append(
          model.predict_proba(X_test, iteration_range=(0,model.best_iteration))[:, 1]
      )


    mean_thr_seed = np.mean(fold_thrs)
    all_fold_thrs.extend(fold_thrs)
    print(f'[seed={sd}] fold mean thr={mean_thr_seed:.3f}')
    oof_preds_seeds.append(oof_preds.values)
    test_preds_seeds.append(np.mean(test_fold_preds, axis=0))

oof_seed_avg = np.mean(oof_preds_seeds, axis=0)
test_seed_avg = np.mean(test_preds_seeds, axis=0)
best_t = np.mean(all_fold_thrs)
best_f1 = f1_score(oof_true, (oof_seed_avg > best_t).astype(int))

print(f'[SEED平均]Best OOF F1: {best_f1:.5f} at thr={best_t:.3f}')




[seed=314] fold mean thr=0.286
[seed=2718] fold mean thr=0.291
[seed=15] fold mean thr=0.284


In [ ]:
test_pred_label = (test_seed_avg > best_t).astype(int)

In [ ]:
sample_sub = pd.read_csv('/content/drive/MyDrive/signate mufg/sample_submit.csv',header=None)

In [ ]:
sample_sub.head(3)

In [ ]:
sub_label = sample_sub.copy()
sub_label[1] = test_pred_label
sub_label.to_csv("mufg_submission.csv", index=False)


In [ ]:
sub_proba_test = sample_sub.copy()
sub_proba_test[1] = test_seed_avg
sub_proba_test.to_csv("test_xgb_thr_avg.csv", index=False)

In [ ]:
train_id = train_data['id'].copy()
sub_proba_train = pd.DataFrame({'id': train_id, 'oof': oof_seed_avg})
sub_proba_train.to_csv("train_xgb_thr_avg.csv", index=False)

In [ ]:
sub_proba_train.head(3)

In [ ]:
sub_proba_test.head(3)

In [ ]:
sub_label.head(3)